In [12]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [31]:
load_dotenv(override=True)

api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [32]:
openai = OpenAI()

In [35]:
message = 'This is my first message to openAI'
response = openai.chat.completions.create(model='gpt-4o-mini', messages = [{'role':'user', 'content':message}])
response.choices[0].message.content
                                          

"Welcome! I'm glad you're here. How can I assist you today?"

In [68]:
#using headers to avoid the website to think I am a bot, headers makes the client(me) look legitimate when I scrape
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.content = soup
        self.title = soup.title.string if soup.title else "No title found"
        
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator = "\n", strip=True)
        
    

In [77]:
ed = Website("https://edwarddonner.com")
print(ed.text)

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for more!
December 21, 2

In [78]:
github = Website("https://www.github.com")
print(github.text)


Skip to content
GitHub Copilot is now available for free.
Learn more
Navigation Menu
Toggle navigation
Sign in
Product
GitHub Copilot
Write better code with AI
Security
Find and fix vulnerabilities
Actions
Automate any workflow
Codespaces
Instant dev environments
Issues
Plan and track work
Code Review
Manage code changes
Discussions
Collaborate outside of code
Code Search
Find more, search less
Explore
All features
Documentation
GitHub Skills
Blog
Solutions
By company size
Enterprises
Small and medium teams
Startups
Nonprofits
By use case
DevSecOps
DevOps
CI/CD
View all use cases
By industry
Healthcare
Financial services
Manufacturing
Government
View all industries
View all solutions
Resources
Topics
AI
DevOps
Security
Software Development
View all
Explore
Learning Pathways
White papers, Ebooks, Webinars
Customer Stories
Partners
Executive Insights
Open Source
GitHub Sponsors
Fund open source developers
The ReadME Project
GitHub community articles
Repositories
Topics
Trending
Collectio

In [79]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [80]:
def user_prompt_for(website):
    user_prompt = f"you are looking at the website titled {website.title} \n"
    user_prompt += "\nThe contents of this website is as follows; \
                    please provide a short summary of this website in markdown. \
                    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [81]:
print(user_prompt_for(ed))

you are looking at the website titled Home - Edward Donner 

The contents of this website is as follows;                     please provide a short summary of this website in markdown.                     If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and 

In [82]:
print(user_prompt_for(github))

you are looking at the website titled GitHub · Build and ship software on a single, collaborative platform · GitHub 

The contents of this website is as follows;                     please provide a short summary of this website in markdown.                     If it includes news or announcements, then summarize these too.

Skip to content
GitHub Copilot is now available for free.
Learn more
Navigation Menu
Toggle navigation
Sign in
Product
GitHub Copilot
Write better code with AI
Security
Find and fix vulnerabilities
Actions
Automate any workflow
Codespaces
Instant dev environments
Issues
Plan and track work
Code Review
Manage code changes
Discussions
Collaborate outside of code
Code Search
Find more, search less
Explore
All features
Documentation
GitHub Skills
Blog
Solutions
By company size
Enterprises
Small and medium teams
Startups
Nonprofits
By use case
DevSecOps
DevOps
CI/CD
View all use cases
By industry
Healthcare
Financial services
Manufacturing
Government
View all industries

In [83]:
def messages_for(website):
    return [
        {'role':'system',  'content':system_prompt},
        {'role':'user', 'content':user_prompt_for(website)}
    ]

In [84]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'you are looking at the website titled Home - Edward Donner \n\nThe contents of this website is as follows;                     please provide a short summary of this website in markdown.                     If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI

In [87]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model= 'gpt-4o-mini',
        messages= messages_for(website)
    )
    return response.choices[0].message.content
    

In [88]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nEdward Donner's website presents his interests in coding, experimenting with Large Language Models (LLMs), and his endeavors in various tech-related fields. He is the co-founder and CTO of Nebula.io, a startup focused on utilizing AI to enhance talent discovery and management. The site highlights his previous experience as the CEO of the AI startup untapt.\n\n## Recent Posts\n- **December 21, 2024:** Welcome, SuperDataScientists!\n- **November 13, 2024:** Mastering AI and LLM Engineering – Resources\n- **October 16, 2024:** From Software Engineer to AI Data Scientist – Resources\n- **August 6, 2024:** Outsmart LLM Arena – A battle of diplomacy and deviousness\n\nThe website also promotes engagement and connection, inviting visitors to reach out and subscribe to a newsletter."

In [89]:
summarize("https://github.com")

"# GitHub Overview\n\nGitHub is a comprehensive platform designed for software development, allowing teams to collaborate and automate their processes effectively. It features a wide array of tools including:\n\n- **GitHub Copilot:** An AI-powered coding assistant that aids in writing code more efficiently, offering features like code completion and chat assistance.\n- **Security Tools:** To help find and fix vulnerabilities quickly, enhancing the overall security of software projects.\n- **GitHub Actions:** For automating workflows and implementing CI/CD practices seamlessly.\n- **GitHub Codespaces:** An instant development environment that allows developers to start projects quickly.\n- **Collaboration Features:** Such as GitHub Discussions and Issues for effective planning and tracking of work.\n\nThe platform supports integration with over 17,000 applications and offers specialized features for various industries and team sizes, from startups to enterprises.\n\n## Recent Announceme

In [90]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


In [91]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website showcases his interests and projects, particularly in the field of AI and large language models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on using AI to enhance talent discovery and management. The site includes a section called "Outsmart," which features an arena where LLMs compete in diplomacy and strategy.

## About the Author
- Edward Donner enjoys coding, experimenting with LLMs, DJing, and electronic music production.
- He was previously the founder and CEO of the AI startup untapt, acquired in 2021.
- He emphasizes the use of proprietary LLMs within talent management and has patented his matching model.

## News and Announcements
- **December 21, 2024:** Welcomes "SuperDataScientists."
- **November 13, 2024:** Lists resources for mastering AI and LLM engineering.
- **October 16, 2024:** Provides resources for transitioning from software engineer to AI data scientist.
- **August 6, 2024:** Highlights the Outsmart LLM arena event.

The website serves as a platform for sharing insights, resources, and an invitation to connect with Donner.

In [92]:
display_summary("https://github.com")

# GitHub Overview

GitHub is a collaborative platform for building and shipping software, integrating development, security, and project management tools into a single solution. The platform offers various features, including:

- **GitHub Copilot**: An AI-powered tool that provides coding assistance, allowing developers to write better code and work faster.
- **GitHub Actions**: A tool for automating workflows and continuous integration/continuous deployment (CI/CD).
- **GitHub Codespaces**: Instant development environments in the cloud for quick setup and coding.
- **Security Tools**: Features that help identify and fix vulnerabilities in code, contributing to secure development practices.

## Key Announcements
- **GitHub Copilot Availability**: GitHub Copilot is now available for free, enhancing productivity through AI-assisted coding.

## Additional Features
- **Collaboration Tools**: Includes code review, project management through GitHub Issues, and discussion threads for team communication.
- **Integration Options**: Supports over 17,000 integrations and has a marketplace for third-party extensions.
- **Customized Workflows**: Offers user-defined project management workflows to fit various development processes.

GitHub aims to improve the developer experience, enhance collaboration and security, and streamline software development for teams of all sizes and industries.